# Demo to test and show the functionality of the library

In [1]:
# Importing Functions
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TKAgg') # Needed to have figures display properly. 
import flirimageextractor
import Utils as u
import cv2

DEBUG=true

In [2]:
filename = "C:\\Users\\Francisco\\Documents\\dataset_termal_destino\\Control_Image termal C_paciente 3 (VC)_flir_control_flir_20190907T092339.jpg"
#filename = "D:\\FLIROne\FLIR_20230426_044637_061.jpg"
#filename = "D:\\FLIROne\FLIR_20230426_044706_939.jpg"
flir = flirimageextractor.FlirImageExtractor(exiftool_path="C:\\Windows\\exiftool.exe")
flir.process_image(filename, RGB=True)

In [3]:
therm = flir.get_thermal_np()
rgb_fullres = flir.get_rgb_np()
if DEBUG:
    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.imshow(therm)
    plt.title('Raw Thermal Image')
    plt.subplot(1,2,2)
    plt.imshow(rgb_fullres)
    plt.title('RGB Full Resolution Image')
    plt.show(block='TRUE') 

In [4]:
colormap_image, Imsalida,temp_img_resized,image_copy = u.extract_images(flir,plot=0)

Image with offsetx=-1, offsety=37, pipx2=479, pipy2=639, real2ir=1.22885632514954


In [5]:
#u.save_thermal_csv(temp_img_resized,"temperature_examples.csv",delimiter=";")

# second part of the demo get the perspective transformation between two images

In [6]:
filename2 = "C:\\Users\\Francisco\\Documents\\dataset_termal_destino\\Control_Image termal C_paciente 3 (VC)_flir_control_flir_20190907T092409.jpg"
flir2 = flirimageextractor.FlirImageExtractor(exiftool_path="C:\\Windows\\exiftool.exe")
flir2.process_image(filename2, RGB=True)
therm2 = flir2.get_thermal_np()
rgb_fullres2 = flir2.get_rgb_np()
colormap_image2, Imsalida2,temp_img_resized2,image_copy2 = u.extract_images(flir2,plot=0)
#u.save_thermal_csv(temp_img_resized2,"temperature_examples2.csv",delimiter=";")
if DEBUG:
    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.imshow(temp_img_resized2)
    plt.title('temp_img_resized2')
    plt.subplot(1,2,2)
    plt.imshow(image_copy2)
    plt.title('image_copy2')
    plt.show(block='TRUE') 


Image with offsetx=-1, offsety=37, pipx2=479, pipy2=639, real2ir=1.22885632514954


In [7]:


"""
def fethearing_bin_to_color(binary_image, color_image):
    # Operación de cierre morfológico con un kernel elíptico de tamaño 5x5
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    closed_image = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel)
    
    # Fethearing usando filtro guiado
    filtered_image = cv2.ximgproc.guidedFilter(color_image, closed_image, 20, eps=0.1)#5, eps=1e-4
    
    # Umbralización de la imagen filtrada
    _, thresholded_image = cv2.threshold(filtered_image,20, 255, cv2.THRESH_BINARY)
    
    return filtered_image
"""


'\ndef fethearing_bin_to_color(binary_image, color_image):\n    # Operación de cierre morfológico con un kernel elíptico de tamaño 5x5\n    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))\n    closed_image = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel)\n    \n    # Fethearing usando filtro guiado\n    filtered_image = cv2.ximgproc.guidedFilter(color_image, closed_image, 20, eps=0.1)#5, eps=1e-4\n    \n    # Umbralización de la imagen filtrada\n    _, thresholded_image = cv2.threshold(filtered_image,20, 255, cv2.THRESH_BINARY)\n    \n    return filtered_image\n'

In [8]:
import SimpleITK as sitk
import numpy as np
import sys

def sitk_image_to_opencv(image, force_8bits=True):
    image_np = sitk.GetArrayFromImage(image)
    return image_np.astype(np.uint8)

def command_iteration(method):
    print(
        f"{method.GetOptimizerIteration():3} "
        + f"= {method.GetMetricValue():10.5f} "
        + f": {method.GetOptimizerPosition()}"
    )

# tomado de https://simpleitk.readthedocs.io/en/master/link_ImageRegistrationMethod4_docs.html

def register_images(fixed_image_np, moving_image_np, output_transform_file,
                    number_of_bins=24, sampling_percentage=0.10):
    
        # Convertir las imágenes a escala de grises
    image_a_gray = np.dot(fixed_image_np, [0.2989, 0.5870, 0.1140]).astype(np.float32)
    image_b_gray = np.dot(moving_image_np, [0.2989, 0.5870, 0.1140]).astype(np.float32)

    fixed = sitk.GetImageFromArray(image_a_gray.astype(np.float32))
    moving = sitk.GetImageFromArray(image_b_gray.astype(np.float32))

    # Convertir imágenes a escala de grises si son multicanal
    if fixed.GetNumberOfComponentsPerPixel() > 1:
        fixed = sitk.Cast(sitk.RescaleIntensity(fixed), sitk.sitkFloat32)
    if moving.GetNumberOfComponentsPerPixel() > 1:
        moving = sitk.Cast(sitk.RescaleIntensity(moving), sitk.sitkFloat32)

    # Verificar el tamaño de las imágenes
    
    if fixed.GetSize()[0] < 4 or moving.GetSize()[0] < 4:
        print(fixed.GetSize())
        print(moving.GetSize())
        raise ValueError("Las imágenes en la dirección 0 deben tener al menos 4 píxeles")

    R = sitk.ImageRegistrationMethod()
    R.SetMetricAsMattesMutualInformation(number_of_bins)
    R.SetMetricSamplingPercentage(sampling_percentage, sitk.sitkWallClock)
    R.SetMetricSamplingStrategy(R.RANDOM)
    R.SetOptimizerAsRegularStepGradientDescent(0.5, 0.001, 400)
    R.SetInitialTransform(sitk.TranslationTransform(fixed.GetDimension()))
    R.SetInterpolator(sitk.sitkLinear)

    R.AddCommand(sitk.sitkIterationEvent, lambda: command_iteration(R))

    outTx = R.Execute(fixed, moving)

    print("-------")
    print(outTx)
    print(f"Optimizer stop condition: {R.GetOptimizerStopConditionDescription()}")
    print(f" Iteration: {R.GetOptimizerIteration()}")
    print(f" Metric value: {R.GetMetricValue()}")

    sitk.WriteTransform(outTx, output_transform_file)

    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(fixed)
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetDefaultPixelValue(100)
    resampler.SetTransform(outTx)

    out = resampler.Execute(moving)

    fixed_np = sitk.GetArrayFromImage(fixed)
    moving_np = sitk.GetArrayFromImage(out)



    simg1 = sitk.Cast(sitk.RescaleIntensity(fixed), sitk.sitkUInt8)
    simg2 = sitk.Cast(sitk.RescaleIntensity(out), sitk.sitkUInt8)
    cimg = sitk.Compose(simg1, simg2, simg1 // 2.0 + simg2 // 2.0)


    return fixed_np, moving_np, sitk_image_to_opencv(cimg)

def register_images_Similarity2DTransform(fixed_image, moving_image, output_transform_file, numberOfBins=24, samplingPercentage=0.10):
    
        
        # Convertir las imágenes a escala de grises
    image_a_gray = np.dot(fixed_image, [0.2989, 0.5870, 0.1140]).astype(np.float32)
    image_b_gray = np.dot(moving_image, [0.2989, 0.5870, 0.1140]).astype(np.float32)

    fixed_image = sitk.GetImageFromArray(image_a_gray.astype(np.float32))
    moving_image = sitk.GetImageFromArray(image_b_gray.astype(np.float32))

    # Convertir imágenes a escala de grises si son multicanal
    if fixed_image.GetNumberOfComponentsPerPixel() > 1:
        fixed_image = sitk.Cast(sitk.RescaleIntensity(fixed_image), sitk.sitkFloat32)
    if moving_image.GetNumberOfComponentsPerPixel() > 1:
        moving_image = sitk.Cast(sitk.RescaleIntensity(moving_image), sitk.sitkFloat32)

    # Verificar el tamaño de las imágenes
    
    if fixed_image.GetSize()[0] < 4 or moving_image.GetSize()[0] < 4:
        print(fixed_image.GetSize())
        print(moving_image.GetSize())
        raise ValueError("Las imágenes en la dirección 0 deben tener al menos 4 píxeles")

    
    
    R = sitk.ImageRegistrationMethod()

    R.SetMetricAsMattesMutualInformation(numberOfBins)
    R.SetMetricSamplingPercentage(samplingPercentage, sitk.sitkWallClock)
    R.SetMetricSamplingStrategy(R.RANDOM)
    R.SetOptimizerAsRegularStepGradientDescent(1.0, 0.001, 200)

    initial_transform = sitk.CenteredTransformInitializer(fixed_image, moving_image, sitk.Similarity2DTransform())
    R.SetInitialTransform(initial_transform)

    R.SetInterpolator(sitk.sitkLinear)

    R.AddCommand(sitk.sitkIterationEvent, lambda: command_iteration(R))

    outTx = R.Execute(fixed_image, moving_image)

    print("-------")
    print(outTx)
    print(f"Optimizer stop condition: {R.GetOptimizerStopConditionDescription()}")
    print(f"Iteration: {R.GetOptimizerIteration()}")
    print(f"Metric value: {R.GetMetricValue()}")

    sitk.WriteTransform(outTx, output_transform_file)

    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(fixed_image)
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetDefaultPixelValue(100)
    resampler.SetTransform(outTx)

    out = resampler.Execute(moving_image)
    simg1 = sitk.Cast(sitk.RescaleIntensity(fixed_image), sitk.sitkUInt8)
    simg2 = sitk.Cast(sitk.RescaleIntensity(out), sitk.sitkUInt8)
    cimg = sitk.Compose(simg1, simg2, simg1 // 2.0 + simg2 // 2.0)

    return {
        "fixed": fixed_image,
        "moving": moving_image,
        "composition": cimg
    }   
    
"""
def register_images(image_a, image_b):
    # Convertir las imágenes a escala de grises
    image_a_gray = np.dot(image_a, [0.2989, 0.5870, 0.1140]).astype(np.uint8)
    image_b_gray = np.dot(image_b, [0.2989, 0.5870, 0.1140]).astype(np.uint8)

    # Crear imágenes ITK a partir de las imágenes en escala de grises
    itk_image_a = itk.GetImageFromArray(image_a_gray)
    itk_image_b = itk.GetImageFromArray(image_b_gray)

    # Definir tipos de imágenes
    ImageType = itk.Image[itk.UC, 2]

    # Crear un objeto de registro de imágenes
    registration_filter = itk.ImageRegistrationMethod[ImageType, ImageType].New()

    # Configurar los parámetros de registro
    optimizer = itk.RegularStepGradientDescentOptimizer.New()
    metric = itk.MeanSquaresImageToImageMetric[ImageType, ImageType].New()
    interpolator = itk.LinearInterpolateImageFunction[ImageType, itk.D].New()

    registration_filter.SetOptimizer(optimizer)
    registration_filter.SetMetric(metric)
    registration_filter.SetInterpolator(interpolator)

    transform = itk.IdentityTransform[itk.D, 2].New()
    registration_filter.SetInitialTransform(transform)

    # Configurar las imágenes de entrada para el registro
    registration_filter.SetFixedImage(itk_image_a)
    registration_filter.SetMovingImage(itk_image_b)

    # Definir la región de muestreo
    region_size = itk.ImageRegion[2]()
    region_size.SetSize([32, 32])

    fixed_image_sampler = itk.RegionOfInterestImageFilter[ImageType, ImageType].New()
    fixed_image_sampler.SetInput(itk_image_a)
    fixed_image_sampler.SetRegionOfInterest(region_size)

    moving_image_sampler = itk.RegionOfInterestImageFilter[ImageType, ImageType].New()
    moving_image_sampler.SetInput(itk_image_b)
    moving_image_sampler.SetRegionOfInterest(region_size)

    registration_filter.SetFixedImageRegion(fixed_image_sampler.GetOutput().GetLargestPossibleRegion())

    # Realizar el registro de las imágenes
    registration_filter.Update()

    # Obtener las imágenes registradas
    registered_image_a = itk.ResampleImageFilter[ImageType, ImageType].New()
    registered_image_a.SetInput(itk_image_b)
    registered_image_a.SetSize(itk_image_a.GetLargestPossibleRegion().GetSize())
    registered_image_a.SetOutputSpacing(itk_image_a.GetSpacing())
    registered_image_a.SetOutputOrigin(itk_image_a.GetOrigin())
    registered_image_a.SetOutputDirection(itk_image_a.GetDirection())
    registered_image_a.SetTransform(registration_filter.GetTransform())

    registered_image_a.Update()

    registered_image_a_array = itk.GetArrayFromImage(registered_image_a)
    registered_image_b_array = image_b

    return registered_image_a_array, registered_image_b_array
def register_images(image_a, image_b):
    # Convertir las imágenes a formato ITK
    itk_image_a = itk.GetImageFromArray(image_a)
    itk_image_b = itk.GetImageFromArray(image_b)

    # Definir los tipos de imágenes y transformaciones
    PixelType = itk.UC
    Dimension = 2
    ImageType = itk.Image[PixelType, Dimension]
    TransformType = itk.TranslationTransform[itk.D, Dimension]

    # Crear un registrador de imágenes
    registration_filter = itk.ImageRegistrationMethod.New(ImageType, ImageType)

    # Configurar los parámetros del registrador
    optimizer = itk.RegularStepGradientDescentOptimizerv4.New()
    metric = itk.MeanSquaresImageToImageMetricv4.New(ImageType, ImageType)
    interpolator = itk.LinearInterpolateImageFunction.New(ImageType, itk.D)

    registration_filter.SetOptimizer(optimizer)
    registration_filter.SetMetric(metric)
    registration_filter.SetInterpolator(interpolator)

    # Definir las transformaciones iniciales y estimar la transformación final
    initial_transform = TransformType.New()
    identity_transform = TransformType.New()
    identity_transform.SetIdentity()

    registration_filter.SetMovingInitialTransform(initial_transform)
    registration_filter.SetFixedInitialTransform(identity_transform)
    registration_filter.SetMovingImage(itk_image_b)
    registration_filter.SetFixedImage(itk_image_a)

    registration_filter.Update()

    # Obtener la imagen registrada
    registered_image = itk.GetArrayFromImage(registration_filter.GetOutput())

    return registered_image
"""


'\ndef register_images(image_a, image_b):\n    # Convertir las imágenes a escala de grises\n    image_a_gray = np.dot(image_a, [0.2989, 0.5870, 0.1140]).astype(np.uint8)\n    image_b_gray = np.dot(image_b, [0.2989, 0.5870, 0.1140]).astype(np.uint8)\n\n    # Crear imágenes ITK a partir de las imágenes en escala de grises\n    itk_image_a = itk.GetImageFromArray(image_a_gray)\n    itk_image_b = itk.GetImageFromArray(image_b_gray)\n\n    # Definir tipos de imágenes\n    ImageType = itk.Image[itk.UC, 2]\n\n    # Crear un objeto de registro de imágenes\n    registration_filter = itk.ImageRegistrationMethod[ImageType, ImageType].New()\n\n    # Configurar los parámetros de registro\n    optimizer = itk.RegularStepGradientDescentOptimizer.New()\n    metric = itk.MeanSquaresImageToImageMetric[ImageType, ImageType].New()\n    interpolator = itk.LinearInterpolateImageFunction[ImageType, itk.D].New()\n\n    registration_filter.SetOptimizer(optimizer)\n    registration_filter.SetMetric(metric)\n   

In [9]:
segmented_Feet2,segmented_temps2=u.Find_feets(image_copy2,temp_img_resized2,percentage=20)
print(segmented_Feet2[1].shape)
if DEBUG:
    u.plot_images_and_thermal(segmented_Feet2,segmented_temps2)
segmented_Feet,segmented_temps=u.Find_feets(image_copy,temp_img_resized,percentage=20)
if DEBUG:
    u.plot_images_and_thermal(segmented_Feet,segmented_temps)


(441, 204, 3)


In [10]:
#u.visualize_descriptors(segmented_Feet[0],segmented_Feet2[0])
#u.visualize_descriptors(segmented_Feet[1],segmented_Feet2[1])

"""
output_image,homography_matrix=u.matchbydescriptors(segmented_Feet[0],segmented_Feet2[0])
plt.figure(figsize=(10,5))
plt.imshow(output_image)
plt.title('registration between images')
plt.show(block='TRUE') 

output_image,homography_matrix=u.matchbydescriptors(segmented_Feet[1],segmented_Feet2[1])
plt.figure(figsize=(10,5))
plt.imshow(output_image)
plt.title('registration between images')
plt.show(block='TRUE') 
"""

"\noutput_image,homography_matrix=u.matchbydescriptors(segmented_Feet[0],segmented_Feet2[0])\nplt.figure(figsize=(10,5))\nplt.imshow(output_image)\nplt.title('registration between images')\nplt.show(block='TRUE') \n\noutput_image,homography_matrix=u.matchbydescriptors(segmented_Feet[1],segmented_Feet2[1])\nplt.figure(figsize=(10,5))\nplt.imshow(output_image)\nplt.title('registration between images')\nplt.show(block='TRUE') \n"

In [11]:
# using a different approach
registered_image_a_array, registered_image_b_array, composition_image =register_images(segmented_Feet[0],segmented_Feet2[0],'displaceMeth4.hdf5')
if DEBUG:
    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.imshow(registered_image_a_array)
    plt.title('registered_image_a_array')
    plt.subplot(1,2,2)
    plt.imshow(registered_image_b_array)
    plt.title('registered_image_b_array')
    plt.show(block='TRUE') 
    
plt.figure(figsize=(10,5))
plt.imshow(composition_image)
plt.title('composition_image')
plt.show(block='TRUE') 


  0 =   -1.15376 : (-0.26354799637605875, -0.4249028754976423)
  1 =   -1.17568 : (-0.5493762076481952, -0.8351494277573426)
  2 =   -1.18581 : (-1.045167812764034, -0.8998849226319477)
  3 =   -1.19082 : (-1.0723113474944264, -0.6513628340314774)
  4 =   -1.19051 : (-1.0872170322705998, -0.7754709343366282)
  5 =   -1.19135 : (-1.0918992754687136, -0.9003832099488902)
  6 =   -1.19090 : (-1.0849754375839202, -0.8382679101512952)
  7 =   -1.19130 : (-1.0805588421018382, -0.7759241559785823)
  8 =   -1.19133 : (-1.0878267471434853, -0.8063172482251044)
  9 =   -1.19135 : (-1.0858249082938252, -0.7908210142884348)
-------
itk::simple::TranslationTransform
 TranslationTransform (0000023DA135B6D0)
   RTTI typeinfo:   class itk::TranslationTransform<double,2>
   Reference Count: 2
   Modified Time: 20531
   Debug: Off
   Object Name: 
   Observers: 
     none
   Offset: [-1.08582, -0.790821]

Optimizer stop condition: RegularStepGradientDescentOptimizerv4: Gradient magnitude tolerance met a

In [12]:
# using a different approach
output =register_images_Similarity2DTransform(segmented_Feet[0],segmented_Feet2[0],'displaceMeth4_advanced.hdf5')
fixed_imagen = sitk_image_to_opencv(output["fixed"])
moving_imagen = sitk_image_to_opencv(output["moving"])
composition_image = sitk_image_to_opencv(output["composition"])
if DEBUG:
    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.imshow(fixed_imagen)
    plt.title('fixed_imagen')
    plt.subplot(1,2,2)
    plt.imshow(moving_imagen)
    plt.title('moving_imagen')
    plt.show(block='TRUE') 

plt.figure(figsize=(10,5))
plt.imshow(composition_image)
plt.title('composition_image')
plt.show(block='TRUE') 


  0 =   -1.18809 : (0.01590953059412137, 0.17724557846697167, -0.29417318528592595, -1.3062772696034124)
  1 =   -0.00204 : (0.5159062677949771, 0.17861454554016515, -0.29509502274367044, -1.3055431488010603)
  2 =   -0.13737 : (0.9310814450683073, 0.4572329804982702, -0.2939987936077246, -1.3059148465769466)
  3 =   -0.28748 : (1.3305938867857743, 0.1566203653854759, -0.29847406457855785, -1.3045596721482824)
  4 =   -0.27400 : (1.1724825525879419, 0.3502681433239908, -0.29960857516138684, -1.3047983840968682)
  5 =   -0.28845 : (0.9785307492476378, 0.19259133205777768, -0.2991852908068798, -1.3093307124519502)
  6 =   -0.52619 : (1.1067854513199684, -0.021997616605314685, -0.30034460290786186, -1.3083455711123857)
  7 =   -0.65926 : (0.9819989338893178, -0.029296639016045168, -0.30013958914985095, -1.3084310622617918)
  8 =   -1.01737 : (1.0162278977010892, 0.02299675849813954, -0.30018406537092257, -1.3082678609699572)
  9 =   -1.06530 : (0.9948970158321728, 0.0001592054326879297, -

 10 =   -1.18138 : (1.008271803038363, 0.008234978085639956, -0.3003777074405622, -1.3081258529231112)
 11 =   -1.14120 : (1.0014233834397903, 0.004475730085305204, -0.30040273109908516, -1.3080842171166769)
 12 =   -1.17932 : (0.9944124242833766, 0.0010292549781012097, -0.3004154399056595, -1.3080271576848763)
 13 =   -1.17994 : (0.9981001546784226, 0.002316527756054485, -0.30045555066440466, -1.3079997074002343)
 14 =   -1.19067 : (0.9963190147505736, 0.0015168381686597918, -0.3004819448307985, -1.307954566123779)
-------
itk::simple::Similarity2DTransform
 Similarity2DTransform (0000023DA121FA30)
   RTTI typeinfo:   class itk::Similarity2DTransform<double>
   Reference Count: 3
   Modified Time: 39813
   Debug: Off
   Object Name: 
   Observers: 
     none
   Matrix: 
     0.996318 -0.00151125 
     0.00151125 0.996318 
   Offset: [0.562521, -1.23171]
   Center: [193.314, 100.048]
   Translation: [-0.300482, -1.30795]
   Inverse: 
     1.00369 0.00152244 
     -0.00152244 1.00369 
 

In [13]:

print(type(fixed_imagen))

print(fixed_imagen.shape)

print(fixed_imagen)

<class 'numpy.ndarray'>
(206, 445)
[[40 40 40 ... 13 13 13]
 [40 40 40 ... 13 13 13]
 [39 39 40 ... 13 13 13]
 ...
 [44 45 43 ... 18 17 16]
 [44 45 43 ... 17 16 16]
 [45 45 43 ... 18 17 16]]
